## 1. Import các thư viện cần thiết

In [1]:
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# #
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add debian buster
# cat > /etc/apt/sources.list.d/debian.list <<'EOF'
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer debian repo for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500


# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300


# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# apt-get update
# apt-get install chromium chromium-driver

# # Install selenium
# pip install selenium

In [2]:
import pandas as pd
import os
import requests
import time
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 2. Crawl

In [3]:
datasets = []

In [4]:
WEBDRIVER_DELAY_TIME_INT = 10

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(4)
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18004\2240435721.py:6: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


In [5]:
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'
for page_idx in tqdm(range(1, 10)):
    main_url = f'https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Age[]=3&Page={page_idx}' 
    driver.get(main_url)

    content_tags_xpath = '//*[@id="PageContent"]/div[2]/div/div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    for idx in range(len(content_tags)):
        content_tag_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]'
        content_title_xpath = f'/html/body/div[4]/div[2]/div/div[{2+idx}]/h4/a'
        content_tag = wait.until(EC.presence_of_element_located((By.XPATH, content_tag_xpath)))
        poem_title = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).text
        poem_url = wait.until(EC.presence_of_element_located((By.XPATH, content_title_xpath))).get_attribute('href')

        try:
            driver.get(poem_url) 
            
            poem_src_xpath = '//span[@class="small"]'
            poem_content_tag = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'poem-content')))

            try:
                poem_content_i_tag = poem_content_tag.find_element(By.TAG_NAME, 'i')
                driver.execute_script(deletion_script, poem_content_i_tag)
            except:
                pass

            try:
                poem_content_b_tag = poem_content_tag.find_element(By.TAG_NAME, 'b')
                driver.execute_script(deletion_script, poem_content_b_tag)
            except:
                pass

            poem_content = poem_content_tag.text

            try:
                poem_src_tag = wait.until(EC.presence_of_element_located((By.XPATH, poem_src_xpath)))
                poem_src = poem_src_tag.text
            except:
                poem_src = ''

            poem_info = {
                'title': poem_title,
                'content': poem_content,
                'source': poem_src,
                'url': poem_url
            }

            datasets.append(poem_info)

            driver.back()
        except Exception as e:
            print(e)
            print(poem_url)

100%|██████████| 9/9 [20:45<00:00, 138.42s/it]


In [10]:
datasets[0]

{'title': '“Cái làm ta hạnh phúc”',
 'content': 'Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều\nChỉ cần có ai đó\nĐể ta thầm thương yêu\n\nRồi thêm chút công việc\nCho ta làm hàng ngày\nCuối cùng, chút mơ mộng\nĐể đưa ta lên mây',
 'source': 'Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB Lao động, 2013',
 'url': 'https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/C%C3%A1i-l%C3%A0m-ta-h%E1%BA%A1nh-ph%C3%BAc/poem-t24-M5Dn2cxmEDVSrQhtCw'}

In [7]:
df = pd.DataFrame(datasets)
df.to_csv('poem_dataset.csv', index=True)

In [13]:
len(set(df['content']))

90

In [9]:
!zip -r poem_dataset.zip poem_dataset.csv

'zip' is not recognized as an internal or external command,
operable program or batch file.
